In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,
KNeighborsClassifier)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/baileyrusso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/baileyrusso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/baileyrusso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
postmates = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/postmates_all7.csv')
postmates = postmates.loc[:, ~postmates.columns.str.contains('^Unnamed')]

In [4]:
postmatesSub = postmates[['Name','MenuItemPrice','Category','MenuItem','MenuItemDescription']]

# Split Menu Item Description By Word

In [5]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [6]:
postmatesSub['MenuItemDescription'] = list(map(lambda row: tokenizer.tokenize(str(row)), postmatesSub['MenuItemDescription']))


/Users/baileyrusso/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


# Split Menu Item Name By Word

In [7]:
postmatesSub['MenuItem'] = list(map(lambda row: tokenizer.tokenize(str(row)), postmatesSub['MenuItem']))


# BOW Preprocessing

In [8]:
#combine columns into bag of words
postmates['BagOfWords'] = postmatesSub['MenuItem'] + postmatesSub['MenuItemDescription']



In [9]:
#lowercase all words
postmates['BagOfWords'] = list(map(lambda row: [str(each_word).lower() for each_word in row], postmates['BagOfWords']))


In [10]:
#remove stopwords
stop_words = stopwords.words('english')
postmates['BagOfWords'] = list(map(lambda word_list: [word for word in word_list if word not in stopwords.words('english')], postmates['BagOfWords']))




In [11]:
#lemmatize words
lemmatizer = WordNetLemmatizer()
postmates['BagOfWords'] = list(map(lambda words: [lemmatizer.lemmatize(word) for word in words], postmates['BagOfWords']))




In [12]:
postmates['BagOfWords'] = list(map(lambda row: ' '.join(row), postmates['BagOfWords']))

# Vectorize BOW

In [13]:
count = CountVectorizer()
countVec = count.fit(postmates['BagOfWords'])
count_matrix = countVec.transform(postmates['BagOfWords'])

# Get Most Similar Items To User Input

In [14]:
def getMostSimilarItems(user_input):
    user_input = user_input.split(' ')
    user_input = [word for word in user_input if word not in stopwords.words('english')]
    user_input = [lemmatizer.lemmatize(word) for word in user_input]
    user_input = ' '.join(user_input)
    user_input = [user_input]
    
    count_matrix_inp = countVec.transform(user_input)
    
    cosine_sim = cosine_similarity(count_matrix, count_matrix_inp)
    most_similar_scores = sorted(cosine_sim[:,0], reverse=True)
    most_similar = cosine_sim[:,0].argsort(axis=0)[::-1]
    most_similar_comb = pd.concat([pd.Series(most_similar), pd.DataFrame(most_similar_scores, columns=['Cosine_Similarity_Score'])], axis=1)
    most_similar_comb = most_similar_comb.set_index(0)

    similar = pd.merge(postmates, most_similar_comb, left_index=True, right_index=True)
    similar = similar.sort_values(by='Cosine_Similarity_Score', ascending=False)
    
    return similar[['Name','MenuItem', 'Cosine_Similarity_Score']]


In [15]:
most_similar = getMostSimilarItems('Chicken parm')
most_similar.head(25)

,Name,MenuItem,Cosine_Similarity_Score
81427,Fumo,Chicken Parm Hero,0.816497
53450,Andys Deli,Chicken Parm Hero,0.707107
61265,Marinara Pizza,Chicken Parm Slice,0.707107
76418,Little Italy Pizza,Chicken Parm Hero,0.707107
44426,B & W Deli & Pizzeria,Chicken Parm Pizza,0.707107
76550,Marinara Pizza,Chicken Parm Slice,0.707107
76597,Marinara Pizza,Chicken Parm Hero,0.707107
21270,B & W Deli & Pizzeria,Chicken Parm Pizza,0.707107
69922,Parm,Chicken Parm Roll,0.707107
69925,Parm,Chicken Parm Platter,0.707107


In [40]:
#postmates.to_csv('postmates_BOW.csv')